In [7]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc

from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First lets look at the GOS

In [26]:
nviews = 305 

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=1.39*50/nviews)

phantom.reconstruct('FDK',filt='ram_lak')

slice_recon = phantom.img[5]

(19, 41, 41) 19
bowtie is off False
really doing it!
ratio is 435.46450211168496 number of photons 0.010763158668836276


In [71]:
# np.save('recons/6x_gos_recon_ram_lak_360',phantom.img[4])
# slice_recon = np.load('recons/6x_gos_recon_ram_lak_360.npy')

In [27]:
plt.figure()
plt.imshow((slice_recon))

recon_slice = slice_recon
im = create_mask(recon_slice.shape,r=5.75)
contrast = []
CNR_fc = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR_fc.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))
# Get rid of the delrin
cc = np.array(sorted(contrast))
star_data = np.array(star_data_gos[1::2])
CNR_fc = np.array(CNR_fc)[np.argsort(contrast)]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
recon_slice = np.rot90(bcca_recon.T[45],2)
im = create_mask(recon_slice.shape,off =[-0.2,0], r= 5.8)
contrast = []
CNR = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))

bcca_data = np.array(sorted(contrast))
CNR_data2 = np.array(CNR)[np.argsort(contrast)]

In [29]:
plt.figure()
plt.plot(CNR_data,CNR_fc)
plt.figure()
plt.plot(CNR_data)
plt.plot(CNR_data2)
plt.plot(CNR_fc,'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/xcite/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets load the experimental data which was generated in ~/MV_CBCT and saved as a npy file

In [23]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_6x_100.npy')

plt.figure()
plt.imshow(np.rot90(bcca_recon.T[45],2),vmax=0.012)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
recon_slice = np.rot90(bcca_recon.T[45],2)
im = create_mask(recon_slice.shape,off =[-0.2,0], r= 5.8)
contrast = []
CNR = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))
    CNR.append(contrast[-1]/np.sqrt(np.std(recon_slice[im == ii])**2 + np.std(recon_slice[im == 3])**2))

bcca_data = np.array(sorted(contrast))
CNR_data = np.array(CNR)[np.argsort(contrast)]

In [5]:
plt.figure()
plt.plot(CNR)
# plt.plot(np.array(contrast)[np.argsort(contrast)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

# bcca_data = bcca_data[np.arange(len(contrast)) !=6 ]
# cc = cc[np.arange(len(contrast)) != 6 ]

res = minimize(HU_fun,[cc[0],cc[3]],args=(cc,bcca_data))


x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
fc_data = (cc - res.x[0])*(res.x[1])

plt.figure()
plt.plot(x,fc_data,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,bcca_data,'s',c='darkorange',label='Star-lack')

# fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

# xx = np.linspace(0,max(x),100)
# pp = np.polyfit(x,fc_data,1,full=1)
# qq = np.polyfit(x,bcca_data,1,full=1)
# yy = np.polyval(pp[0],xx)
# zz = np.polyval(qq[0],xx)

# plt.ylabel('HU')
# plt.xlabel('Relative electron density')
# plt.title('CWO')
# plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
# plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
# plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
# plt.legend()
# plt.tight_layout()
# plt.savefig('initial_cwo')

NameError: name 'cc' is not defined

In [16]:
plt.figure()
scale, offset, offset2 = 19.8, 0.00650, 0.057
plt.plot(scale*(bcca_data - offset),'rx')
plt.plot(cc - offset2,'kx')
plt.ylabel('Machine Output Value [??]')
plt.xlabel('Material')
plt.legend(['BCCA data','fastCAT'])
plt.title('Experimental Validation 6 MV 100MU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Experimental Validation 6 MV 100MU')

In [10]:
1-pp[1]/np.std(fc_data)**2

array([0.99978516])

In [11]:
1-qq[1]/np.std(star_data)**2

array([1.])

# Now lets look at the kV data 

In [27]:
nviews = 360

In [28]:
s = fc.calculate_spectrum(100, 12, 3, 50,monitor=None)
s.attenuate(0.2,fc.get_mu(z=13))
kernel = fc.Kernel(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

(19, 19, 19) 19


In [29]:
phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_Acrylic','CATPHAN_Delrin','CATPHAN_Acrylic','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_Acrylic','CATPHAN_LDPE','CATPHAN_Acrylic','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=7000/nviews,bowtie=True)
phantom.reconstruct('FDK',filt='ram_lak')

bowtie is off False
Using the bowtie filter True
really doing it!
ratio is 360822.99832277594 number of photons 0.7874215885840448


In [31]:
# np.save('recons/kV_CsI_recon_ram_lak_360',phantom.img[4])
# slice_recon = np.load('recons/.npy')
slice_recon = phantom.img[4]

In [32]:
# np.save('recon_no_bowtie',slice_recon)

In [33]:
plt.figure()
plt.imshow(np.rot90(slice_recon))

recon_slice = slice_recon
im = create_mask(recon_slice.shape,r=5.75)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure()
plt.imshow(np.rot90(slice_recon))

recon_slice = slice_recon
im = create_mask(recon_slice.shape,r=5.75)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets load the experimental kV data which was generated in ~/MV_CBCT and saved as a npy file

In [35]:
bcca_recon = np.load('/home/xcite/MV_CBCT/jericho_data/dicom_volume.npy')

plt.figure()
plt.imshow(np.rot90(bcca_recon.T[50],2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
recon_slice = np.rot90(bcca_recon.T[50])
im = create_mask(recon_slice.shape,off = [-0.1,-0.02], r= 4.45, radius = 0.3)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

bcca_data = np.array(sorted(contrast))[np.arange(len(contrast)) ]

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

In [37]:
def HU_fun(a,x,y):
    return np.sum(np.abs((x - a[0])*(a[1]) - y))

# bcca_data = bcca_data[np.arange(len(contrast)) !=6 ]
# cc = cc[np.arange(len(contrast)) != 6 ]

res = minimize(HU_fun,[cc[0],6600],args=(cc,bcca_data),method = 'COBYLA')

# x = [0,0,.853,0.945,.998,1.147,1.87]
            
fc_data = (cc - res.x[0])*(res.x[1])
x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,bcca_data,'s',c='darkorange',label='')
plt.plot(x,fc_data,'r*',c='cornflowerblue',label='fastCAT')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
res

In [218]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 +1000 - y))

bcca_revised = bcca_data
cc2 = cc

res = minimize(HU_fun,[cc2[3],0.1*cc2[0]],args=(cc2,bcca_revised))

x = [0,0,.853,0.945,.998,1.147,1.87]
            
fc_data = (cc2 - res.x[0])/(res.x[1] - res.x[0])*-1000 + 1000
x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,bcca_revised,'s',c='darkorange',label='Star-lack')
plt.plot(x,fc_data,'r*',c='cornflowerblue',label='fastCAT')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [163]:
res

      fun: 18.501660223129925
 hess_inv: array([[ 3.73296625e-05, -1.30796365e-04],
       [-1.30796365e-04,  4.58785464e-04]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 42
      nit: 3
     njev: 14
   status: 0
  success: True
        x: array([0.2180453 , 0.01511759])

In [68]:
fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,bcca_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

In [172]:
plt.figure()
scale, offset, offset2 = 16.5, 0.00650, 0.057
plt.plot(scale*(bcca_data - offset),'rx')
plt.plot(cc - offset2,'kx')
plt.xlabel('Machine Output Value [??]')
plt.ylabel('Material')
plt.legend(['BCCA data','fastCAT'])
plt.title('Experimental Validation 6 MV 100MU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Experimental Validation 6 MV 100MU')

In [153]:
1-pp[1]/np.std(fc_data)**2

array([0.99967224])

In [154]:
1-qq[1]/np.std(star_data)**2

array([1.])